In [1]:
from modules.statistical import *

modules.preprocessing successfull loaded: 100%|███| 6/6 [00:01<00:00,  3.49it/s]


In [2]:
import numpy as np

In [3]:
layers = list(range(1,14))
layers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [4]:
#analyzer_launcher(outputs_name="outputs", analytical_func=compare_MlC_MCA_for_GLO_PER_GAP, layers=layers, approach=approach, aggregation_f=None,logics=None, alphaConfig=[0.1,0.3,0.4,0.5,0.6,0.7,0.8,0.9])

[
    analyzer_launcher_for_descriptor_rank_plot(
        outputs_name="outputs", 
        analytical_func=Descriptors_comparison, 
        approach=['mixed'],
        layers=layers, 
        aggregation_f=statistics.mean, 
        top=k, 
        alphaConfig=[0.1,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
        with_class=False
    )  for k in [10,15,20,15,25,30]
]

In [5]:
alphas = [round(i,1) for i in np.arange(0.1,1.0,0.1)]
#alphas.append(0.85)
#alphas = sorted(list(set(alphas)-set([0.2, 0.4])))
alphas

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [6]:
generate_report_tables(
    outputs_name='outputs310524', 
    cwd=os.getcwd(), 
    layers=layers, 
    _type='qualitative',
    alphas=alphas
)

['GERMAN', 'JAPAN']
GERMAN
{'LDA': 'LDA', 'LR': 'LR', 'SVM': 'SVM', 'DT': 'DT', 'RF': 'RF', 'XGB': 'XGB'}
GERMAN [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] // 0.1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
here
here
here
here
here
here
JAPAN
{'LDA': 'LDA', 'LR': 'LR', 'SVM': 'SVM', 'DT': 'DT', 'RF': 'RF', 'XGB': 'XGB'}
JAPAN [1, 2, 3, 4, 5, 6, 7, 8, 9] // 0.1 [1, 2, 3, 4, 5, 6, 7, 8, 9]
Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze/ig/tableaux/0_1' created successfully.
Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze/ig/tableaux/0_1' already exists.
Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze/ig/tableaux/0_1' already exists.
Directory '/Users/djiemboutienctheuvictornico/Documents/MyFolders/ACADEMIC/M2_thesis/scripting/analyze/ig/tableaux/0_1' already exists.
Directory '/Users/djiemboutienctheuvictornico/Documents/My

In [8]:
data = joblib.load('analyze/best.tex')

In [9]:
data

{'GERMAN': {0.1: {'real_best_k': [1],
   'predicted_best_k': [3],
   'value': [0.315],
   'model': ['LDA']},
  0.2: {'real_best_k': [10],
   'predicted_best_k': [3],
   'value': [0.295],
   'model': ['LDA']},
  0.3: {'real_best_k': [1],
   'predicted_best_k': [3],
   'value': [0.305],
   'model': ['XGB']},
  0.4: {'real_best_k': [1],
   'predicted_best_k': [3],
   'value': [0.305],
   'model': ['XGB']},
  0.5: {'real_best_k': [2],
   'predicted_best_k': [1],
   'value': [0.31],
   'model': ['LDA']},
  0.6: {'real_best_k': [2],
   'predicted_best_k': [2],
   'value': [0.305],
   'model': ['RF']},
  0.7: {'real_best_k': [3],
   'predicted_best_k': [3],
   'value': [0.305],
   'model': ['XGB']},
  0.8: {'real_best_k': [1],
   'predicted_best_k': [3],
   'value': [0.31],
   'model': ['DT']},
  0.9: {'real_best_k': [1],
   'predicted_best_k': [3],
   'value': [0.295],
   'model': ['DT']}},
 'JAPAN': {0.1: {'real_best_k': [8],
   'predicted_best_k': [1],
   'value': [0.3623188405797101],
   

In [11]:
df = pd.DataFrame(data)
df

,GERMAN,JAPAN
0.1,"{'real_best_k': [1], 'predicted_best_k': [3], ...","{'real_best_k': [8], 'predicted_best_k': [1], ..."
0.2,"{'real_best_k': [10], 'predicted_best_k': [3],...","{'real_best_k': [2], 'predicted_best_k': [6], ..."
0.3,"{'real_best_k': [1], 'predicted_best_k': [3], ...","{'real_best_k': [3], 'predicted_best_k': [5], ..."
0.4,"{'real_best_k': [1], 'predicted_best_k': [3], ...","{'real_best_k': [2], 'predicted_best_k': [2], ..."
0.5,"{'real_best_k': [2], 'predicted_best_k': [1], ...","{'real_best_k': [2], 'predicted_best_k': [2], ..."
0.6,"{'real_best_k': [2], 'predicted_best_k': [2], ...","{'real_best_k': [2], 'predicted_best_k': [5], ..."
0.7,"{'real_best_k': [3], 'predicted_best_k': [3], ...","{'real_best_k': [1], 'predicted_best_k': [2], ..."
0.8,"{'real_best_k': [1], 'predicted_best_k': [3], ...","{'real_best_k': [7], 'predicted_best_k': [1], ..."
0.9,"{'real_best_k': [1], 'predicted_best_k': [3], ...","{'real_best_k': [3], 'predicted_best_k': [1], ..."


In [14]:
df.index = [alphas,list(data['GERMAN'][0.1].keys())]

ValueError: all arrays must be same length